교통 표지판 탐지 시스템 실습 가이드

In [ ]:
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import cv2
import torch
from pathlib import Path
import requests
import shutil

In [ ]:
# MLflow 비활성화 (에러 방지)
def nuclear_mlflow_disable():
    """MLflow 관련 모든 것을 완전히 비활성화"""
    
    print("🚫 Completely disabling MLflow...")
    
    # 1. 모든 MLflow 환경변수 제거/설정
    mlflow_vars = [
        'MLFLOW_TRACKING_URI',
        'MLFLOW_REGISTRY_URI', 
        'MLFLOW_DEFAULT_ARTIFACT_ROOT',
        'MLFLOW_BACKEND_STORE_URI',
        'MLFLOW_ARTIFACT_URI',
        'MLFLOW_S3_ENDPOINT_URL',
        'MLFLOW_EXPERIMENT_ID',
        'MLFLOW_RUN_ID'
    ]
    
    for var in mlflow_vars:
        if var in os.environ:
            del os.environ[var]
        os.environ[var] = ''
    
    # 2. 추가 비활성화 플래그들
    os.environ['DISABLE_MLFLOW'] = '1'
    os.environ['NO_MLFLOW'] = '1'
    os.environ['MLFLOW_DISABLE'] = '1'
    os.environ['MLFLOW_DISABLE_ENV_CREATION'] = '1'
    
    # 3. runs 폴더 완전 삭제
    runs_paths = [
        Path('./runs'),
        Path('./mlflow'),
        Path('C:/Users/JPJ/deeplearning-curriculum/runs'),
        Path('C:/Users/JPJ/deeplearning-curriculum/mlflow'),
    ]
    
    for runs_path in runs_paths:
        if runs_path.exists():
            try:
                shutil.rmtree(runs_path, ignore_errors=True)
                print(f"🗑️ Removed: {runs_path}")
            except Exception as e:
                print(f"⚠️ Could not remove {runs_path}: {e}")
    
    # 4. 현재 작업 디렉토리 변경 (MLflow 설정 회피)
    original_dir = os.getcwd()
    temp_dir = Path.home() / 'temp_yolo_training'
    temp_dir.mkdir(exist_ok=True)
    os.chdir(temp_dir)
    print(f"📁 Changed working directory to: {temp_dir}")
    
    # 5. MLflow import 완전 차단
    class MLflowBlocker:
        def find_spec(self, name, path, target=None):
            if 'mlflow' in name.lower():
                print(f"🚫 Blocked MLflow import: {name}")
                return None
        def find_module(self, name, path=None):
            if 'mlflow' in name.lower():
                print(f"🚫 Blocked MLflow import: {name}")
                return None
    
    # MLflow 차단기 설치
    if not any(isinstance(finder, MLflowBlocker) for finder in sys.meta_path):
        sys.meta_path.insert(0, MLflowBlocker())
    
    print("✅ MLflow completely neutralized")
    return original_dir    

1단계: Roboflow로 데이터셋 다운로드

In [ ]:
def download_traffic_dataset_roboflow():
    """Roboflow에서 교통표지판 데이터셋 다운로드"""
    
    try:
        from roboflow import Roboflow
        
        print("🔄 Downloading traffic sign dataset from Roboflow...")
        
        # Roboflow API 사용
        rf = Roboflow(api_key="qsoKXZgHZOA51nr36zzD")  # 실제 API 키로 교체
        project = rf.workspace("augmented-startups").project("traffic-sign-detection-yolov8")
        dataset = project.version(1).download("yolov8")
        
        print("✅ Dataset downloaded successfully!")
        return dataset.location
        
    except Exception as e:
        print(f"❌ Roboflow download failed: {e}")
        print("📥 Creating sample dataset instead...")
        return create_sample_dataset()

def create_sample_dataset():
    """샘플 데이터셋 생성 (Roboflow 실패 시 대안)"""
    print("🔧 Creating sample dataset...")
    
    dataset_name = "traffic-signs-sample"
    base_path = Path(dataset_name)
    
    # 디렉토리 구조 생성
    for split in ['train', 'valid', 'test']:
        (base_path / split / 'images').mkdir(parents=True, exist_ok=True)
        (base_path / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    # 샘플 이미지 다운로드
    sample_images = [
        "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/bus.jpg",
        "https://raw.githubusercontent.com/ultralytics/yolov5/master/data/images/zidane.jpg",
    ]
    
    for i, url in enumerate(sample_images):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            
            # 이미지 저장
            img_name = f"sample_{i:03d}.jpg"
            for split in ['train', 'valid']:
                img_path = base_path / split / 'images' / img_name
                with open(img_path, 'wb') as f:
                    f.write(response.content)
                
                # 더미 라벨 생성 (YOLO 포맷)
                label_path = base_path / split / 'labels' / f"sample_{i:03d}.txt"
                with open(label_path, 'w') as f:
                    f.write("0 0.5 0.5 0.3 0.3\n")  # class x_center y_center width height
                    
            print(f"✅ Downloaded sample image {i+1}")
            
        except Exception as e:
            print(f"❌ Failed to download sample {i}: {e}")
    
    # data.yaml 생성
    data_yaml_content = f"""
path: {base_path.absolute()}
train: train/images
val: valid/images
test: test/images

nc: 5
names: ['speed_limit_30', 'speed_limit_50', 'stop_sign', 'no_entry', 'yield']
"""
    
    with open(base_path / 'data.yaml', 'w') as f:
        f.write(data_yaml_content.strip())
    
    print(f"✅ Sample dataset created at: {base_path}")
    return str(base_path)

2단계: YOLOv11 모델 훈련

In [ ]:
def train_yolov11_model(dataset_path, model_size='n', epochs=50):

    from ultralytics import YOLO
    
    print(f"🚀 Training YOLOv11{model_size} model...")
    
    # 디바이스 설정
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"💻 Using device: {device}")
    
    # GPU 메모리에 따른 배치 크기 자동 조정
    if device == 'cuda':
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)  # GB
        if gpu_memory < 6:
            batch_size = 8
        elif gpu_memory < 12:
            batch_size = 16
        else:
            batch_size = 32
        print(f"🔧 GPU Memory: {gpu_memory:.1f}GB, Batch size: {batch_size}")
    else:
        batch_size = 4
        print(f"🔧 CPU mode, Batch size: {batch_size}")
    
    # YOLOv11 모델 로드
    try:
        model = YOLO(f'yolo11{model_size}.pt')
        print(f"✅ YOLOv11{model_size} model loaded successfully")
    except Exception as e:
        print(f"❌ Failed to load YOLOv11{model_size}: {e}")
        print("🔄 Trying YOLOv8 as fallback...")
        model = YOLO(f'yolov8{model_size}.pt')
        print(f"✅ YOLOv8{model_size} fallback loaded")
    
    # MLflow 관련 설정 제거
    print(f"🏃‍♂️ Starting training for {epochs} epochs...")
    start_time = time.time()
    
    try:
        results = model.train(
            data=f'{dataset_path}/data.yaml',
            epochs=epochs,
            batch=batch_size,
            imgsz=640,
            device=device,
            patience=15,
            save=True,
            project='traffic_signs_yolov11',
            name=f'yolo11{model_size}_experiment',
            verbose=True,
            
            # 성능 최적화
            cache='disk',             # RAM 대신 disk 캐시 (MLflow 에러 방지)
            amp=device=='cuda',       # Automatic Mixed Precision (GPU만)
            workers=4 if device=='cuda' else 2,  # 데이터 로더 워커
            
            # 데이터 증강
            hsv_h=0.015,
            hsv_s=0.7,
            hsv_v=0.4,
            degrees=10,
            translate=0.1,
            scale=0.5,
            flipud=0.0,
            fliplr=0.5,
            
            # 학습 설정
            optimizer='AdamW',
            lr0=0.001,
            weight_decay=0.0005,
            
            # MLflow 비활성화
            plots=False,              # plots 비활성화로 MLflow 우회
        )
        
        training_time = time.time() - start_time
        print(f"✅ Training completed in {training_time/60:.1f} minutes")
        
        return model, results
        
    except Exception as e:
        print(f"❌ Training failed: {e}")
        return None, None

3단계: 모델 성능 평가

In [ ]:
def evaluate_model_performance(model, dataset_path):
    """모델 성능 종합 평가"""
    print("📊 Evaluating model performance...")
    
    try:
        # 검증 데이터로 평가
        val_results = model.val(data=f'{dataset_path}/data.yaml', verbose=False)
        
        # 성능 지표 추출
        performance_metrics = {
            'mAP50': float(val_results.box.map50) if hasattr(val_results.box, 'map50') else 0.0,
            'mAP50_95': float(val_results.box.map) if hasattr(val_results.box, 'map') else 0.0,
            'precision': float(val_results.box.mp) if hasattr(val_results.box, 'mp') else 0.0,
            'recall': float(val_results.box.mr) if hasattr(val_results.box, 'mr') else 0.0,
        }
        
        print(f"📈 Performance Metrics:")
        print(f"   mAP@0.5: {performance_metrics['mAP50']:.3f}")
        print(f"   mAP@0.5:0.95: {performance_metrics['mAP50_95']:.3f}")
        print(f"   Precision: {performance_metrics['precision']:.3f}")
        print(f"   Recall: {performance_metrics['recall']:.3f}")
        
        return performance_metrics
        
    except Exception as e:
        print(f"❌ Evaluation failed: {e}")
        return {'mAP50': 0.0, 'mAP50_95': 0.0, 'precision': 0.0, 'recall': 0.0}

def measure_inference_speed(model, dataset_path, num_samples=20):
    """추론 속도 측정"""
    print("⏱️ Measuring inference speed...")
    
    try:
        # 테스트 이미지 수집
        test_dir = Path(dataset_path) / 'test' / 'images'
        if not test_dir.exists():
            test_dir = Path(dataset_path) / 'valid' / 'images'
        
        test_images = list(test_dir.glob('*.jpg'))[:num_samples]
        
        if not test_images:
            print("❌ No test images found")
            return {'fps': 0.0, 'avg_time': 0.0}
        
        # 추론 시간 측정
        times = []
        
        for img_path in test_images:
            start_time = time.time()
            results = model.predict(source=str(img_path), verbose=False, save=False)
            end_time = time.time()
            times.append(end_time - start_time)
        
        avg_time = np.mean(times)
        fps = 1.0 / avg_time if avg_time > 0 else 0.0
        
        print(f"🚀 Inference Speed:")
        print(f"   Average time per image: {avg_time:.3f}s")
        print(f"   FPS: {fps:.1f}")
        
        return {
            'fps': fps,
            'avg_time': avg_time,
            'total_images': len(test_images)
        }
        
    except Exception as e:
        print(f"❌ Speed measurement failed: {e}")
        return {'fps': 0.0, 'avg_time': 0.0}

4단계: 결과 시각화

In [ ]:
def visualize_training_results(project_path):
    """훈련 결과 시각화"""
    print("📈 Visualizing training results...")
    
    results_path = Path(project_path) / "results.csv"
    
    if not results_path.exists():
        print(f"❌ Results file not found: {results_path}")
        return
    
    try:
        df = pd.read_csv(results_path)
        
        # 그래프 스타일 설정
        plt.style.use('seaborn-v0_8')
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. Loss 곡선
        if 'train/box_loss' in df.columns and 'val/box_loss' in df.columns:
            axes[0,0].plot(df['epoch'], df['train/box_loss'], 
                          label='Train Box Loss', color='#2E86AB', linewidth=2)
            axes[0,0].plot(df['epoch'], df['val/box_loss'], 
                          label='Val Box Loss', color='#A23B72', linewidth=2)
            axes[0,0].set_title('Box Loss During Training', fontsize=14, fontweight='bold')
            axes[0,0].set_xlabel('Epoch')
            axes[0,0].set_ylabel('Loss')
            axes[0,0].legend()
            axes[0,0].grid(True, alpha=0.3)
        
        # 2. mAP 곡선
        map_cols = [col for col in df.columns if 'mAP' in col and 'val' in col]
        if map_cols:
            for i, col in enumerate(map_cols[:2]):
                color = ['#F18F01', '#C73E1D'][i]
                axes[0,1].plot(df['epoch'], df[col], 
                              label=col.replace('val/', ''), color=color, linewidth=2)
            axes[0,1].set_title('Mean Average Precision', fontsize=14, fontweight='bold')
            axes[0,1].set_xlabel('Epoch')
            axes[0,1].set_ylabel('mAP')
            axes[0,1].legend()
            axes[0,1].grid(True, alpha=0.3)
        
        # 3. 정밀도/재현율
        precision_cols = [col for col in df.columns if 'precision' in col and 'val' in col]
        recall_cols = [col for col in df.columns if 'recall' in col and 'val' in col]
        
        if precision_cols and recall_cols:
            axes[1,0].plot(df['epoch'], df[precision_cols[0]], 
                          label='Precision', color='#3D5A80', linewidth=2)
            axes[1,0].plot(df['epoch'], df[recall_cols[0]], 
                          label='Recall', color='#98C1D9', linewidth=2)
            axes[1,0].set_title('Precision and Recall', fontsize=14, fontweight='bold')
            axes[1,0].set_xlabel('Epoch')
            axes[1,0].set_ylabel('Score')
            axes[1,0].legend()
            axes[1,0].grid(True, alpha=0.3)
        
        # 4. 학습률
        lr_cols = [col for col in df.columns if 'lr' in col]
        if lr_cols:
            axes[1,1].plot(df['epoch'], df[lr_cols[0]], 
                          label='Learning Rate', color='#E63946', linewidth=2)
            axes[1,1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
            axes[1,1].set_xlabel('Epoch')
            axes[1,1].set_ylabel('Learning Rate')
            axes[1,1].legend()
            axes[1,1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        save_path = Path(project_path) / 'training_analysis.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"✅ Training visualization saved to: {save_path}")
        plt.show()
        
    except Exception as e:
        print(f"❌ Visualization failed: {e}")

def visualize_detection_results(model, dataset_path, num_samples=6):
    """샘플 이미지에서 탐지 결과 시각화"""
    print("🎯 Visualizing detection results...")
    
    try:
        # 테스트 이미지 찾기
        test_dir = Path(dataset_path) / 'test' / 'images'
        if not test_dir.exists():
            test_dir = Path(dataset_path) / 'valid' / 'images'
        
        test_images = list(test_dir.glob('*.jpg'))[:num_samples]
        
        if not test_images:
            print("❌ No test images found")
            return
        
        # 그리드 크기 계산
        cols = min(3, len(test_images))
        rows = (len(test_images) + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
        if rows == 1 and cols == 1:
            axes = [axes]
        elif rows == 1:
            axes = axes
        else:
            axes = axes.flatten()
        
        for i, img_path in enumerate(test_images):
            # 예측 수행
            results = model.predict(source=str(img_path), save=False, verbose=False)
            
            # 이미지 로드
            img = cv2.imread(str(img_path))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # 바운딩 박스 그리기
            if len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes.xyxy.cpu().numpy()
                confs = results[0].boxes.conf.cpu().numpy()
                classes = results[0].boxes.cls.cpu().numpy() if results[0].boxes.cls is not None else []
                
                for box, conf in zip(boxes, confs):
                    if conf > 0.3:  # 신뢰도 임계값
                        x1, y1, x2, y2 = box.astype(int)
                        
                        # 바운딩 박스
                        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (255, 0, 0), 3)
                        
                        # 신뢰도 텍스트
                        text = f'{conf:.2f}'
                        cv2.putText(img_rgb, text, (x1, y1-10), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
            
            # 결과 표시
            if i < len(axes):
                axes[i].imshow(img_rgb)
                axes[i].set_title(f'Detection Result {i+1}', fontweight='bold')
                axes[i].axis('off')
        
        # 빈 서브플롯 숨기기
        for j in range(len(test_images), len(axes)):
            axes[j].axis('off')
        
        plt.tight_layout()
        plt.savefig('detection_results.png', dpi=300, bbox_inches='tight')
        print("✅ Detection results saved to: detection_results.png")
        plt.show()
        
    except Exception as e:
        print(f"❌ Detection visualization failed: {e}")

def compare_model_sizes(dataset_path, model_sizes=['n', 's'], epochs=10):
    """여러 모델 크기 비교"""
    print("🏆 Comparing different model sizes...")
    
    comparison_results = []
    
    for size in model_sizes:
        print(f"\n{'='*50}")
        print(f"Training YOLOv11{size}")
        print(f"{'='*50}")
        
        # 모델 훈련
        model, results = train_yolov11_model(dataset_path, size, epochs)
        
        if model is None:
            continue
        
        # 성능 평가
        performance = evaluate_model_performance(model, dataset_path)
        speed = measure_inference_speed(model, dataset_path)
        
        # 모델 크기 정보
        model_sizes_mb = {'n': 6, 's': 22, 'm': 52, 'l': 131, 'x': 218}
        
        comparison_results.append({
            'Model': f'YOLOv11{size}',
            'mAP50': performance['mAP50'],
            'mAP50_95': performance['mAP50_95'],
            'Precision': performance['precision'],
            'Recall': performance['recall'],
            'FPS': speed['fps'],
            'Avg_Time': speed['avg_time'],
            'Model_Size_MB': model_sizes_mb.get(size, 0)
        })
    
    return comparison_results

def visualize_model_comparison(comparison_results):
    """모델 비교 결과 시각화"""
    if not comparison_results:
        print("❌ No comparison results to visualize")
        return
    
    df = pd.DataFrame(comparison_results)
    
    # 그래프 스타일 설정
    plt.style.use('seaborn-v0_8')
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
    
    # 1. mAP 비교
    bars1 = axes[0,0].bar(df['Model'], df['mAP50'], 
                         color=colors[:len(df)], alpha=0.8, edgecolor='black')
    axes[0,0].set_title('mAP@0.5 Comparison', fontsize=14, fontweight='bold')
    axes[0,0].set_ylabel('mAP@0.5')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # 값 표시
    for bar, value in zip(bars1, df['mAP50']):
        axes[0,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                      f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 2. FPS 비교
    bars2 = axes[0,1].bar(df['Model'], df['FPS'], 
                         color=colors[:len(df)], alpha=0.8, edgecolor='black')
    axes[0,1].set_title('Inference Speed Comparison', fontsize=14, fontweight='bold')
    axes[0,1].set_ylabel('FPS')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # 값 표시
    for bar, value in zip(bars2, df['FPS']):
        axes[0,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                      f'{value:.1f}', ha='center', va='bottom', fontweight='bold')
    
    # 3. 모델 크기 비교
    bars3 = axes[1,0].bar(df['Model'], df['Model_Size_MB'], 
                         color=colors[:len(df)], alpha=0.8, edgecolor='black')
    axes[1,0].set_title('Model Size Comparison', fontsize=14, fontweight='bold')
    axes[1,0].set_ylabel('Size (MB)')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # 값 표시
    for bar, value in zip(bars3, df['Model_Size_MB']):
        axes[1,0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                      f'{value}MB', ha='center', va='bottom', fontweight='bold')
    
    # 4. 정확도 vs 속도 산점도
    scatter = axes[1,1].scatter(df['FPS'], df['mAP50'], 
                               s=df['Model_Size_MB']*5, 
                               c=range(len(df)), cmap='viridis',
                               alpha=0.7, edgecolors='black', linewidth=2)
    
    # 모델 이름 표시
    for i, model in enumerate(df['Model']):
        axes[1,1].annotate(model, (df['FPS'].iloc[i], df['mAP50'].iloc[i]),
                          xytext=(5, 5), textcoords='offset points',
                          fontweight='bold', fontsize=10)
    
    axes[1,1].set_xlabel('FPS (Higher is Better)')
    axes[1,1].set_ylabel('mAP@0.5 (Higher is Better)')
    axes[1,1].set_title('Accuracy vs Speed Trade-off\n(Bubble size = Model size)', 
                       fontsize=14, fontweight='bold')
    axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
    print("✅ Model comparison saved to: model_comparison.png")
    plt.show()
    
    # 결과 테이블 출력
    print("\n" + "="*80)
    print("📊 MODEL COMPARISON SUMMARY")
    print("="*80)
    print(df.round(3).to_string(index=False))
    print("="*80)


5단계: 실제 이미지 테스트

In [ ]:
def main():
    """전체 실습 메인 함수"""
    print("🚀 YOLOv11 Traffic Sign Detection System")
    print("="*60)
    original_dir = nuclear_mlflow_disable()
    try:
        # 1. 데이터셋 다운로드
        print("\n📥 Step 1: Downloading dataset...")
        dataset_path = download_traffic_dataset_roboflow()
        print(f"✅ Dataset ready at: {dataset_path}")
        
        # 2. 단일 모델 훈련
        print("\n🏃‍♂️ Step 2: Training YOLOv11 model...")
        model, results = train_yolov11_model(dataset_path, model_size='n', epochs=20)
        
        if model is None:
            print("❌ Training failed, exiting...")
            return
        
        # 3. 성능 평가
        print("\n📊 Step 3: Evaluating performance...")
        performance = evaluate_model_performance(model, dataset_path)
        speed = measure_inference_speed(model, dataset_path)
        
        # 4. 결과 시각화
        print("\n📈 Step 4: Visualizing results...")
        visualize_training_results('traffic_signs_yolov11/yolo11n_experiment')
        visualize_detection_results(model, dataset_path)
        
        # 5. 모델 비교 (선택사항)
        compare_models = input("\n❓ Compare multiple model sizes? (y/n): ").lower().strip() == 'y'
        
        if compare_models:
            print("\n🏆 Step 5: Comparing model variants...")
            comparison_results = compare_model_sizes(dataset_path, ['n', 's'], epochs=10)
            if comparison_results:
                visualize_model_comparison(comparison_results)
        
        # 6. 요약 출력
        print("\n" + "="*60)
        print("🎉 TRAINING COMPLETED SUCCESSFULLY!")
        print("="*60)
        print(f"📊 Final Performance:")
        print(f"   mAP@0.5: {performance['mAP50']:.3f}")
        print(f"   mAP@0.5:0.95: {performance['mAP50_95']:.3f}")
        print(f"   FPS: {speed['fps']:.1f}")
        print(f"   Avg inference time: {speed['avg_time']:.3f}s")
        print("\n📁 Generated files:")
        print("   - training_analysis.png: Training process visualization")
        print("   - detection_results.png: Sample detection results")
        print("   - model_comparison.png: Model comparison (if selected)")
        print("   - trained model: traffic_signs_yolov11/yolo11n_experiment/weights/best.pt")
        print("="*60)
        
    except KeyboardInterrupt:
        print("\n⚠️ Training interrupted by user")
    except Exception as e:
        print(f"\n❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()
    